#### Importando as Bibliotecas Necessárias

In [1]:
import os
import pandas as pd
import pdfplumber

#### Função para converter pdf em excel

In [ ]:
def pdf_to_excel(pdf_path, excel_path):
    with pdfplumber.open(pdf_path) as pdf:
        first_table = pdf.pages[0]
        table = first_table.extract_table()

    df = pd.DataFrame(table)
    df.columns = df.iloc[0]
    df.to_excel(excel_path, index=False)

def convert_all_pdfs_in_directory(source_directory, destination_directory):
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    files = os.listdir(source_directory)

    pdf_files = [arq for arq in files if arq.endswith('.pdf')]

    for pdf_file in pdf_files:
        try:
            pdf_path = os.path.join(source_directory, pdf_file)
            excel_path = os.path.join(destination_directory, pdf_file.replace('.pdf', '.xlsx'))
            
            pdf_to_excel(pdf_path, excel_path)
            print(f"Arquivo {pdf_file} convertido com sucesso!")
        except Exception as e:
            print(f"Erro ao converter {pdf_file}: {e}")

source_directory = r'0_pdf'
destination_directory = r'1_excel'

convert_all_pdfs_in_directory(source_directory, destination_directory)

#### ETL do primeiro PDF após conversão para Excel

In [ ]:
def process_and_save_excel_file(excel_path, output_directory):
    df = pd.read_excel(excel_path)
    df = df.iloc[5:].copy()
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)

    company_name = df.iloc[0, 0].split(":")[1].strip()
    cpf_cnpj = df.iloc[0, 5].split(":")[1].strip()
    qualification = df.iloc[1, 0].split(":")[1].strip()

    all_columns = ["Informações", "Valor Mobiliário Derivativo", "Características dos Títulos", "Intermediário", "Operação", "Dia", "Quantidade", "Preço", "Volume (R$)"]
    df_final = pd.DataFrame(columns=all_columns)

    basic_info = {
        "Informações": ["Denominação da Companhia: " + company_name, "Nome: " + company_name, "CPF/CNPJ: " + cpf_cnpj, "Qualificação: " + qualification]
    }
    df_basic_info = pd.DataFrame(basic_info)
    df_final = pd.concat([df_final, df_basic_info], ignore_index=True)

    saldo_inicial = {"Informações": "Saldo Inicial"}
    df_saldo_inicial = pd.DataFrame([saldo_inicial])
    df_final = pd.concat([df_final, df_saldo_inicial], ignore_index=True)

    initial_actions = {"Valor Mobiliário Derivativo": "Ações", "Características dos Títulos": "PN", "Quantidade": df.iloc[5, 8]}
    df_initial_actions = pd.DataFrame([initial_actions])
    df_final = pd.concat([df_final, df_initial_actions], ignore_index=True)

    movements = {"Informações": "Movimentações no Mês"}
    df_movements = pd.DataFrame([movements])
    df_final = pd.concat([df_final, df_movements], ignore_index=True)

    actions_movements = {"Valor Mobiliário Derivativo": "Ações", "Características dos Títulos": "PN", "Intermediário": df.iloc[8, 2], "Operação": df.iloc[8, 3], "Dia": df.iloc[8, 4], "Quantidade": df.iloc[8, 5], "Preço": df.iloc[8, 6], "Volume (R$)": df.iloc[8, 7]}
    df_actions_movements = pd.DataFrame([actions_movements])
    df_final = pd.concat([df_final, df_actions_movements], ignore_index=True)

    final_balance = {"Informações": "Saldo Final"}
    df_final_balance = pd.DataFrame([final_balance])
    df_final = pd.concat([df_final, df_final_balance], ignore_index=True)

    final_actions = {"Valor Mobiliário Derivativo": "Ações", "Características dos Títulos": "PN", "Quantidade": df.iloc[11, 6]}
    df_final_actions = pd.DataFrame([final_actions])
    df_final = pd.concat([df_final, df_final_actions], ignore_index=True)

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    output_path = os.path.join(output_directory, os.path.basename(excel_path))
    df_final.to_excel(output_path, index=False)
    return df_final

input_directory = '1_excel'
input_filename = 'ABCB4 - BCO ABC BRASIL S.A. - Valores Mobiliários Negociados e Detidos 2023-06-01 - prot 1121359.xlsx'
input_path = os.path.join(input_directory, input_filename)
output_directory = '2_excel_processed'

process_and_save_excel_file(input_path, output_directory)

#### Visualização do Dataframe (retirando células vazias)

In [ ]:
df = pd.read_excel('2_excel_processed/ABCB4 - BCO ABC BRASIL S.A. - Valores Mobiliários Negociados e Detidos 2023-06-01 - prot 1121359.xlsx')
df = df.fillna('')
display(df)